In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import heapq
from queue import PriorityQueue
import copy
import sys

In [ ]:
print("Enter the name of file on which you want to run this Alog : ")
inp = input()

In [ ]:
G = nx.read_edgelist(inp, nodetype=int, data=(("weight", int),))
G = nx.Graph(G)
lst = []
for s, d, wt in G.edges(data=True):
    wt = wt.get('weight')
    e = (s, d, wt)
    if wt == 0:
        lst.append(e)

for i in lst:
    G.remove_edge(*i[:2])

In [ ]:
list(G.edges(data=True))
source = list(G.edges())
source, x = source[0]
print("-----------------------------------------------------------------")
print("source is " + str(source))

In [ ]:
def print_graph(inp_graph, ct):
    pos = nx.spring_layout(inp_graph)
    nx.draw_networkx(inp_graph, pos)
    labels = nx.get_edge_attributes(inp_graph, 'weight')
    x = nx.draw_networkx_edge_labels(inp_graph, pos, edge_labels=labels)
    plt.show("filename" + str(ct) + ".png")

In [ ]:
# Initialize
input_graph = copy.deepcopy(G)
print_graph(input_graph, 0)
no_of_nodes = input_graph.number_of_nodes()
# heap = [] # store as (node, g-value)
pq = PriorityQueue()

In [ ]:
print("No of nodes in graph " + str(input_graph.number_of_nodes()))
# print_graph(input_graph, 0)

In [ ]:
def kruskalAlgo(inp_graph):
    # stores the edges present in MST
    MST = []
    index = 0
    cost = 0
    nodes = inp_graph.number_of_nodes()
    # temp graph
    graph = nx.Graph()
    # sort edges by increasing weight
    sorted_graph = sorted(inp_graph.edges(data=True), key=lambda item: item[2]['weight'])
    # MST contains exactly `V-1` edges
    i = 0
    for edgelist in sorted_graph:
        # consider the next edge with minimum weight from the graph
        (src, dest, weight) = edgelist
        weight = weight.get('weight')
        index = index + 1
        # add edge
        graph.add_edge(src, dest, weight=weight)
        # check for cycle
        lst = nx.cycle_basis(graph.to_undirected())
        if len(lst) == 0:
            # consider the wt if no cycle
            cost = cost + weight
        else:
            # discard the edge
            e = (src, dest, {"weight" : weight})
            graph.remove_edge(*e[:2])
    cost = 0
    for s,d,wt in graph.edges(data="weight"):
        cost = cost + wt
    # print(cost)
    return graph, cost

In [ ]:
#works
def get_g_n(inp_graph, src, dest, parent_g_n):
    e = (src, dest)
    if inp_graph.has_edge(*e):
        return inp_graph[src][dest].get('weight') + parent_g_n
    else:
        return -1

In [ ]:
#works
def get_h_n_and_mst(inp_graph):
    graph, cost = kruskalAlgo(inp_graph)
    # print("Cost of MST " + str(cost))
    # print_graph(graph,0)
    return cost

In [ ]:
#works
def remove_list_of_nodes_from_graph(src, inp_graph, lst):
    # lst = visited + [u]
    for i in lst:
        if int(i) != src:
            if int(i) in inp_graph:
                inp_graph.remove_node(int(i))
    return inp_graph

In [ ]:
#works
def check_for_exactly_once_visited(input_graph ,visited):
    lst = list(input_graph)
    for i in lst:
        if i not in visited:
            return False
    return True

In [ ]:
#works
def goal_test(src, visited):
    if src == visited[len(visited)-1] and len(visited)!=1:
        return True
    return False

In [ ]:
def A_star(inp_graph, N, src):
    # state
    count = 1
    # GN
    GN = 0
    # ct = []
    visited = []
    # deepcopy for aug graph
    temp = copy.deepcopy(inp_graph)

    h_n = get_h_n_and_mst(temp)
    # h_n = 0
    pq.put((h_n + GN, h_n, GN, src, copy.deepcopy(visited), src))
    # std bfs jaisa
    while pq.qsize() != 0:
        data = pq.get()
        f_n_parent, h_n_parent, g_n_parent, node, visited, parent = data
        count = max(pq.qsize(), count)
        # make node lst
        visited.append(node)

        if goal_test(src, visited):
            visited = [src] + visited
            print("-----------------------------------------------------------------")
            print("Path Followed is ")
            print(visited)
            return f_n_parent, count

        if check_for_exactly_once_visited(inp_graph , visited):
            print("-----------------------------------------------------------------")
            print("All nodes are visited exactly once")
            print(str(visited + [src]))

            x = visited[len(visited) -1]
            e = (x, src)

            if not inp_graph.has_edge(*e):
                print("No direct edges between " + str(x) + " and " + str(src))
                print("Prune above path :)")
                continue
            else:
                visited.remove(src)

        # get edges of current node
        edges = []
        for u, v, weight in inp_graph.edges.data("weight"):
            if weight is not None:
                if u == node and v not in visited:
                    edges.append(v)
                elif v == node and u not in visited:
                    edges.append(u)
        # push all those childs into heap
        for u in edges:
            temp = copy.deepcopy(inp_graph)

            temp = remove_list_of_nodes_from_graph(src, temp, visited + [u])

            h_n = get_h_n_and_mst(temp)
            # edge wt + parent's gn
            GN = get_g_n(inp_graph, node, u, g_n_parent)
            
            if GN == -1:
                print("Direct edge doesn't exist for " + str(u) + " and " + str(node))
                break
            # push into heap
            pq.put((h_n + GN, h_n, GN, u, copy.deepcopy(visited),node))
            # count = count + 1
        
    if pq.qsize() == 0:
        # print("We haven't traversed all nodes and heap is empty")
        print("Solution does not exist")
        # print("Fail")
        ans = 0
    return ans, count

In [ ]:
def Start_a_star():
    temp = copy.deepcopy(input_graph)
    # print_graph(input_graph, 0)
    path, count = A_star(input_graph, no_of_nodes, source)    
    print("FN = " + str(path))
    print("Max size of FringList during execution = " + str(count))
    print("-----------------------------------------------------------------")

In [ ]:
Start_a_star()